In [ ]:
import json
import os
import sys
from typing import Dict
import yaml
from pydantic import BaseModel

root_dir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(root_dir, "src"))


from config import get_settings
from data_cutter.types.schema import SchemaConfig
from data_cutter.schema_maker import SchemaModelMaker

settings = get_settings()

# Compare JSON Schema

## DataCutter

In [2]:
with open("schemas/example1.json", "r") as f:
    schema_config = json.load(f)
    
config = SchemaConfig.model_validate(schema_config)

In [3]:
config

SchemaConfig(name='Result', fields=[FieldSpec(name='items', specification=DtypeSpec(dim=1, dtype='Item', allowed_values=None, optional=False, description=None, pattern=None, format=None, multipleOf=None, minimum=None, maximum=None, exclusiveMinimum=None, exclusiveMaximum=None, minItems=None, maxItems=None))], custom_dtypes=[CustomDTypeSpec(name='Item', fields=[FieldSpec(name='value', specification=DtypeSpec(dim=0, dtype='string', allowed_values=None, optional=False, description=None, pattern=None, format=None, multipleOf=None, minimum=None, maximum=None, exclusiveMinimum=None, exclusiveMaximum=None, minItems=None, maxItems=None)), FieldSpec(name='label', specification=DtypeSpec(dim=0, dtype='string', allowed_values=['a', 'b'], optional=False, description=None, pattern=None, format=None, multipleOf=None, minimum=None, maximum=None, exclusiveMinimum=None, exclusiveMaximum=None, minItems=None, maxItems=None))])])

In [4]:
model = SchemaModelMaker().make(config)

## Pydantic Answer

In [6]:
from enum import Enum
from typing import List

class Label(str, Enum):
    a="a"
    b="b"

class Item(BaseModel):
    value: str
    label: Label
    
    class Config:
        extra="forbid"

class Result(BaseModel):
    items: List[Item]
    
    class Config:
        extra="forbid"

/var/folders/k4/hpklmk6s2vd2lgvfmg9wcm7h0000gn/T/ipykernel_65081/3485441573.py:8: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class Item(BaseModel):
/var/folders/k4/hpklmk6s2vd2lgvfmg9wcm7h0000gn/T/ipykernel_65081/3485441573.py:15: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class Result(BaseModel):


In [7]:
answer = Result

## compare

In [10]:
print("DataCutter")
print(json.dumps(model.model_json_schema(), indent=2))

DataCutter
{
  "$defs": {
    "Item": {
      "additionalProperties": false,
      "properties": {
        "value": {
          "title": "Value",
          "type": "string"
        },
        "label": {
          "$ref": "#/$defs/Item_label_enum"
        }
      },
      "required": [
        "value",
        "label"
      ],
      "title": "Item",
      "type": "object"
    },
    "Item_label_enum": {
      "enum": [
        "a",
        "b"
      ],
      "title": "Item_label_enum",
      "type": "string"
    }
  },
  "additionalProperties": false,
  "properties": {
    "items": {
      "items": {
        "$ref": "#/$defs/Item"
      },
      "title": "Items",
      "type": "array"
    }
  },
  "required": [
    "items"
  ],
  "title": "Result",
  "type": "object"
}


In [11]:
print("Answer")
print(json.dumps(answer.model_json_schema(), indent=2))

Answer
{
  "$defs": {
    "Item": {
      "additionalProperties": false,
      "properties": {
        "value": {
          "title": "Value",
          "type": "string"
        },
        "label": {
          "$ref": "#/$defs/Label"
        }
      },
      "required": [
        "value",
        "label"
      ],
      "title": "Item",
      "type": "object"
    },
    "Label": {
      "enum": [
        "a",
        "b"
      ],
      "title": "Label",
      "type": "string"
    }
  },
  "additionalProperties": false,
  "properties": {
    "items": {
      "items": {
        "$ref": "#/$defs/Item"
      },
      "title": "Items",
      "type": "array"
    }
  },
  "required": [
    "items"
  ],
  "title": "Result",
  "type": "object"
}
